<a href="https://colab.research.google.com/github/Servetvrll/kredi_risk_siniflandirma/blob/Veri-on-i%C5%9Flem-devam%C4%B1/Kredi_risk_siniflandirma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/credit_risk_dataset/credit_risk_dataset.csv")

print(data.head())
print("---------------------")
print(data.info())
print("---------------------")
print(data.describe())
print("---------------------")
print(data.isnull().sum())
print("---------------------")

# veride 4105 adet person_emp_length de 0 a eşit veri var
# 30 dan büyük 9 değer var
sayac = 0
for i in range(len(data["person_emp_length"])):
  if data["person_emp_length"][i] == 0:
    sayac += 1
    # print(i)
    # print(data["person_emp_length"][i])
print(f"person_emp_length == {sayac}")
print("---------------------")

# person_age sütununda 70 üstü yaşa sahip 7 kişi bulunmakta
degerlendirme = (data["person_age"] > 80).sum()
print(data[data["person_age"] > 80]["person_age"])



plt.figure(figsize=(18,8))
plt.subplot(2,3,1)
sns.distplot(data["person_income"])
plt.subplot(2,3,2)
sns.distplot(data["person_age"])
plt.subplot(2,3,3)
sns.distplot(data["person_emp_length"])
plt.subplot(2,3,4)
sns.distplot(data["loan_amnt"])
plt.subplot(2,3,5)
sns.distplot(data["loan_int_rate"])
plt.subplot(2,3,6)
sns.distplot(data["loan_percent_income"])
plt.show()

## 0 ve 210. indexte 123 değerleri var bunları ay olrak düşünüp 10 yıla eşitliyorum
data.loc[[0, 210], "person_emp_length"] = [10, 10]



# person_age sütununda 70 üstü yaşa sahip 7 kişi bulunmakta 144 144 123 123 144 94 84 yaşlar bu şekilde bunlar ortalamadan çok uzak değerler olduğundan dolayı
# Bu verileri silmeye karar verdim
data.drop(data[data["person_age"] > 80].index, inplace=True)
print(data.describe())
print("---------------------")

# boş verileri doldurma ilk olarak person_emp_length çalışma süresi 0 ile 20 yıl arasında değişkenler içermekte
# Eksik olması çalışmadığı anlamına gelebilir 0 ile doldurabiliriz ya da median ile doldurarak diğer değerleri etkilemeden doldurabiliriz
# şuan sistemde 4105 adet 0 değeri var boş olan 865 adet veri var bunlarıda medianla doldurmayı tercih ediyorum
data["person_emp_length"].fillna(data["person_emp_length"].median(), inplace=True)

# Diğer bir boş veri loan_int_rate Faiz oranı faiz oranını loan_grade (kredi notu) ile orantılı bir şekilde artıp azalmaktadır
# Bu sebeplerden dolayı faiz kategorisinde boş olan 3116 değeri kredi notu gruplarıyla gruplandırıp onların median değerleriyle dolduracağım
# Bu sayede daha tutarlı bir doldurma yapmış olduğumu düşünmekteyim
data['loan_int_rate'] = data.groupby('loan_grade')['loan_int_rate'].transform(
    lambda x: x.fillna(x.median())
)

data.isnull().sum()


In [ ]:
print(data['loan_grade'].value_counts())
print(data['person_home_ownership'].value_counts())
print(data['loan_intent'].value_counts())
print(data['cb_person_default_on_file'].value_counts())

# Kategorik değerler numeric değerlere dönüşümleri yapıldı
from sklearn.preprocessing import LabelEncoder
kategorik_sutunlar = data.select_dtypes(include=['object']).columns
print(kategorik_sutunlar)

for col in kategorik_sutunlar:
    if data[col].nunique() == 2:
        # 2 sınıf varsa Label Encoding
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
    else:
        # 2'den fazla sınıf varsa One-Hot Encoding
        data = pd.get_dummies(data, columns=[col], drop_first=False)



In [ ]:
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns

for col in numerical_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    print(f"{col}: {len(outliers)} aykırı değer")



for col in numerical_cols:
    plt.figure(figsize=(6, 1.5))
    sns.boxplot(x=data[col])
    plt.title(f"{col} için boxplot")
    plt.show()

from scipy.stats import zscore

z_scores = data[numerical_cols].apply(zscore)
outliers = (abs(z_scores) > 3).sum()
print(outliers)

sns.countplot(x='loan_status', data=data)
plt.title("Kredi Durumlarının Dağılımı")
plt.show()

plt.figure(figsize=(14,10))
sns.heatmap(data.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Korelasyon Matrisi")
plt.show()

categorical_columns = [col for col in data.columns if data[col].dtype == 'bool']

for col in categorical_columns:
    oran = data.groupby(col)["loan_status"].mean()
    print(f"\n{col} değişkenine göre kredi reddedilme ortalaması:")
    print(oran)

clip_columns = [
    'person_age', 'person_income', 'person_emp_length',
    'loan_amnt', 'loan_int_rate', 'loan_percent_income',
    'cb_person_cred_hist_length'
]

for col in clip_columns:
    plt.figure(figsize=(10, 4))
    sns.histplot(data[col], kde=True)
    plt.title(f"{col} Dağılımı")
    plt.show()

    sns.boxplot(x=data[col])
    plt.title(f"{col} Boxplot")
    plt.show()

sns.boxplot(x='loan_status', y='person_income', data=data)
plt.title("Kredi Durumuna Göre Gelir Dağılımı")
plt.show()

In [58]:
# Aykırı değerlere uygulanacak sütunlar


# Her bir sütun için IQR hesapla ve clip uygula
for col in clip_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = data[col].clip(lower_bound, upper_bound)

from sklearn.preprocessing import StandardScaler

# Ölçeklenecek sütunlar
scale_columns = clip_columns

# Yeni bir scaler nesnesi
scaler = StandardScaler()

# Sadece scale edilecek kısmı al, uygula
data[scale_columns] = scaler.fit_transform(data[scale_columns])



In [ ]:
sns.countplot(x='loan_status', data=data)
plt.title("Kredi Durumlarının Dağılımı")
plt.show()

plt.figure(figsize=(14,10))
sns.heatmap(data.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Korelasyon Matrisi")
plt.show()

